Question 3

In [11]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
train_datagen.fit(x_train)

test_datagen = ImageDataGenerator()

model = Sequential([
    Flatten(input_shape=(32, 32, 3)),  
    Dense(128, activation='relu'),     
    Dense(64, activation='relu'),      
    Dense(10, activation='softmax')    
    ])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32,
          epochs=10,
          validation_data=test_datagen.flow(x_test, y_test, batch_size=32),
          validation_steps=len(x_test) / 32)

test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_accuracy)

Epoch 1/10
1562/1562 [==============================] - 27s 16ms/step - loss: 2.0068 - accuracy: 0.2641 - val_loss: 1.8454 - val_accuracy: 0.3242
Epoch 2/10
1562/1562 [==============================] - 25s 16ms/step - loss: 1.9037 - accuracy: 0.3083 - val_loss: 1.7922 - val_accuracy: 0.3604
Epoch 3/10
1562/1562 [==============================] - 26s 17ms/step - loss: 1.8527 - accuracy: 0.3296 - val_loss: 1.7358 - val_accuracy: 0.3744
Epoch 4/10
1562/1562 [==============================] - 26s 17ms/step - loss: 1.8264 - accuracy: 0.3405 - val_loss: 1.6758 - val_accuracy: 0.4042
Epoch 5/10
1562/1562 [==============================] - 26s 17ms/step - loss: 1.8018 - accuracy: 0.3509 - val_loss: 1.6588 - val_accuracy: 0.4050
Epoch 6/10
1562/1562 [==============================] - 27s 17ms/step - loss: 1.7859 - accuracy: 0.3554 - val_loss: 1.6522 - val_accuracy: 0.4038
Epoch 7/10
1562/1562 [==============================] - 25s 16ms/step - loss: 1.7758 - accuracy: 0.3606 - val_loss: 1.6299 -

Question 4

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize images
])

svhn_train = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
svhn_test = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

train_loader = DataLoader(svhn_train, batch_size=32, shuffle=True)
test_loader = DataLoader(svhn_test, batch_size=32, shuffle=False)

models_to_compare = {
    "AlexNet": models.alexnet(pretrained=True),
    "VGG16": models.vgg16(pretrained=True),
    "ResNet18": models.resnet18(pretrained=True),
    "ResNet50": models.resnet50(pretrained=True),
    "ResNet101": models.resnet101(pretrained=True)
}

def fine_tune_model(model, num_classes):
    if isinstance(model, models.ResNet):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
    elif isinstance(model, models.VGG):
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    elif isinstance(model, models.AlexNet):
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    else:
        raise NotImplementedError("Model not supported for fine-tuning")

    return model

for model_name, model in models_to_compare.items():
    model = fine_tune_model(model, num_classes=10)


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"{model_name} - Accuracy on test set: {accuracy:.2f}%")
